In [2]:
import os, gc

In [3]:
#load the 40b falcon model from hugging face
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/home/landon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.85s/it]


In [4]:
model.eval()
model.to('cpu')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [5]:
import gc
gc.collect()

72

In [6]:
#perform dynamic quantization on the model
import torch

In [7]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [9]:
gc.collect()

0

In [10]:
save_dir = "models/llama-2-7b-chat-hf-qint8-4th"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [12]:
# Save the state dictionary of the quantized model
quantized_model_path = os.path.join(save_dir, "quantized_model.pth")
torch.save(quantized_model.state_dict(), quantized_model_path)

# Save the tokenizer
tokenizer.save_pretrained(save_dir)

('models/llama-2-7b-chat-hf-qint8-4th/tokenizer_config.json',
 'models/llama-2-7b-chat-hf-qint8-4th/special_tokens_map.json',
 'models/llama-2-7b-chat-hf-qint8-4th/tokenizer.json')

: 